In [1]:
from lerLaudo import lerLaudo
from LerSimuladorExcel import lerSimulador
from lerContrato import lerContrato, dadosParticipantes
#from preencherAPI3 import preencherAPI
from apiAz import preencherAPI

In [2]:
path = r'G:\Drives compartilhados\Pontte Crédito\0_HOME EQUITY\0_Analises\Valquiria Maria Falcao Benevides de Souza Leão - ID 592631549\KIT QI\HE_Contrato_Valquiria_Assinatura Digital_VFinal-Assinado.pdf'
lerContrato(path)

37


{'valorTotal': '396277.74',
 'tabela': 'SAC',
 'registro': '3516.05',
 'Taxa ao Mes': 0.142996,
 'valorLiquido': '365000.00',
 'prazoMes': '240',
 'valorPrimeiraParcelaComEncargos': '6339.15',
 'valorImóvel': '675000.00',
 'prazoContrato': '239',
 'ultimaParcela': '23/12/2042',
 'dataLiberação': '06/12/2022',
 'valorPrimeiraParcela': '6204.48',
 'primeiraParcela': '23/02/2023',
 'indice': 'IPCA',
 'Matrícula': '66.326',
 'Cartório': '02 de Registro Geral do 2º Oficial de Registro de Imóveis da Comarca de Recife/PE',
 'CCI': '0003036552/VMF',
 'Titular': 'VALQUIRIA MARIA FALCÃO BENEVIDES DE SOUZA LEÃO ',
 'operacao': 'PF',
 'Quantidade': 2,
 'Participante1': 'VALQUIRIA MARIA F . B. DE SOUZA LEÃO',
 'Participação1': '100%',
 'dataContrato': '02/12/2022'}

Código 1.1

In [3]:
import bdb
from errno import WSABASEERR
from tkinter import *
import tkinter as tk
import tkinter
from tkinter.font import BOLD
from tkinter.ttk import *
from tkinter.filedialog import askopenfile
from time import sleep
import time
import os
from turtle import bgcolor, width 
import warnings
from pycep_correios import get_address_from_cep, WebService

#Remover Avisos
warnings.filterwarnings('ignore', category=UserWarning, module='tkinter')

#Definir Variáveis Globais
contrato = None
simulador = None
laudo = None
textoContrato = None
botaoUm = None
produto = None

ws = Tk()
ws.title('Validação Contrato x Simulador x Laudo')
ws.geometry('1920x1080') 

#Passo 1 - Criar um main frame
main_frame = Frame(ws)
main_frame.pack(fill=BOTH, expand=1)

#Passo 2 - Criar um canvas
my_canvas = Canvas(main_frame)
my_canvas.pack(side=LEFT, fill=BOTH, expand=1)

#Passo 3 - Adicionar a barra de rolagem ao canvas
scrollbar = Scrollbar(main_frame, orient=VERTICAL, command=my_canvas.yview)
scrollbar.pack(side=RIGHT, fill=Y)

#Passo 4 - Configurar o canvas
my_canvas.configure(yscrollcommand=scrollbar.set)
my_canvas.bind('<Configure>', lambda e:my_canvas.configure(scrollregion=my_canvas.bbox("all")))

#Passo 5 - Criar um outro frame dentro do canvas
canvas = Frame(my_canvas)

#Passo 6 - Adicionar o novo frame dentro de uma janela no canvas
my_canvas.create_window((0,0), window=canvas, anchor='nw')



#Função Selecionar PDF
def open_file_pdf(entryBox):
    file_path = askopenfile(mode='r', filetypes=[('ALL', 'pdf')])
    entryBox.delete(0, 'end')
    if file_path is not None:
        pass
    entryBox.insert(END, file_path.name) # add this

#Função Selecionar Excel (xlsm)
def open_file_excel(entryBox):
    entryBox.delete(0, 'end')
    file_path = askopenfile(mode='r', filetypes=[('ALL','*xlsm *xlsx')])
    if file_path is not None:
        pass
    entryBox.insert(END, file_path.name) # add this

#Função para verificar se o path é válido
def verificarArquivo(arquivo):
    try:
        with open(arquivo, 'r') as f:
            print('Aquivo Existe')
    except IOError:
        print('Arquivo não encontrado!')

#Função para definir paths    
def savePaths():
    global contrato 
    global simulador
    global laudo
    global calculoFluxo
    global produto

    contrato = textBoxContrato.get()
    laudo = textBoxLaudo.get()
    simulador = textBoxSimulador.get()
    calculoFluxo = textBoxCalculoFluxo.get()

    if "HE_" in contrato:
        produto = 'HE'
    else: 
        produto = 'FI'

    if contrato != "" and laudo != "" and simulador != "":
        return 0
    else:
        return 1

#Ler o Contrato
def infosContrato(pathContrato):
    global textBoxContrato
    global textoParticipantes
    global tipoOps
    global valorCCI
    textoContrato = lerContrato(pathContrato)
    textoParticipantes = dadosParticipantes(pathContrato, textoContrato)
    
    #Separar Key-Value
    valorBrutoC = str(textoContrato['valorTotal']).replace(".",",")
    valorLiquidoC = str(textoContrato['valorLiquido']).replace(".",",")
    valorAmortizacaoC = str(textoContrato['tabela'])
    valorIndiceC = str(textoContrato['indice'])
    prazoC = str(textoContrato['prazoMes'])
    parcelasContratoC = str(textoContrato['prazoContrato'])
    dataDesembolcoC = str(textoContrato['dataLiberação'])
    dataPriParC = str(textoContrato['primeiraParcela'])
    dataUltimaParC = str(textoContrato['ultimaParcela'])
    valorPriParC = str(textoContrato['valorPrimeiraParcela']).replace(".",",")
    valorPriParEncargosC = str(textoContrato['valorPrimeiraParcelaComEncargos']).replace(".",",")
    valorAvaliacaoC = str(textoContrato['valorImóvel']).replace(".",",")
    matriculaImovelC = str(textoContrato['Matrícula']).replace(".","")
    cartorioImovelC = str(textoContrato['Cartório'])
    tipoOps = str(textoContrato['operacao'])
    valorCCI = str(textoContrato['CCI'])
    dataContratoC = str(textoContrato['dataContrato'])

    try:
        taxaJuros = float(textoContrato['Taxa ao Mes'])*100
    except:
        taxaJuros = str(textoContrato['Taxa ao Mes'])
        
    #Novas Extrações
    quantidadeC = str(textoContrato['Quantidade'])


    #Clear Box
    tbCValorBruto.delete(0, 'end')
    tbCValorLiquido.delete(0, 'end')
    tbCAmortizacao.delete(0, 'end')
    tbCIndice.delete(0, 'end')
    tbCPrazo.delete(0, 'end')
    tbCParcelas.delete(0, 'end')
    tbCDataDesem.delete(0, 'end')
    tbCDataPriParcela.delete(0, 'end')
    tbCDataUltimaParcela.delete(0, 'end')
    tbCValorPriPar.delete(0, 'end')
    tbCValorPriEncargosPar.delete(0, 'end')
    tbCValorAvaliacao.delete(0, 'end')
    tbCMatricula.delete(0, 'end')
    tbCCartorio.delete(0, 'end')

    #Taxa de Juros
    textBoxTaxaJuros.delete(0, 'end')
    
    #Novos Casos
    tbCNumParticipantes.delete(0, 'end')
    tbCDataContrato.delete(0, 'end')

    #Inserir dados do Contrato
    tbCValorBruto.insert(END, valorBrutoC)
    tbCValorLiquido.insert(END, valorLiquidoC)
    tbCAmortizacao.insert(END, valorAmortizacaoC)
    tbCIndice.insert(END, valorIndiceC)
    tbCPrazo.insert(END, prazoC)
    tbCParcelas.insert(END, parcelasContratoC)
    tbCDataDesem.insert(END, dataDesembolcoC)
    tbCDataPriParcela.insert(END, dataPriParC)
    tbCDataUltimaParcela.insert(END, dataUltimaParC)
    tbCValorPriPar.insert(END, valorPriParC)
    tbCValorPriEncargosPar.insert(END, valorPriParEncargosC)
    tbCValorAvaliacao.insert(END, valorAvaliacaoC)
    tbCMatricula.insert(END, matriculaImovelC)
    tbCCartorio.insert(END, cartorioImovelC)
    #Taxa de Juros
    textBoxTaxaJuros.insert(END, taxaJuros)
    
    #Novos Casos
    tbCDataContrato.insert(END, dataContratoC)
    tbCNumParticipantes.insert(END,quantidadeC)

    return textoContrato

#Ler Simulador
def infosSimulador(pathSimulador):
    textoSimulador = lerSimulador(pathSimulador)
    
    #Separar Key-Value
    valorBrutoS = str(round(textoSimulador['valorTotal'],2)).replace(".",",")
    valorLiquidoS = str(textoSimulador['valorLiquido']).replace(".",",")
    valorAmortizacaoS = str(textoSimulador['tabela'])
    valorIndiceS = str(textoSimulador['indice'])
    prazoS = str(textoSimulador['prazoMes'])
    parcelasContratoS = str(textoSimulador['prazoContrato'])
    dataDesembolcoS = str(textoSimulador['dataContrato'])
    dataPriParS = str(textoSimulador['primeiraParcela'])
    dataUltimaParS = str(textoSimulador['ultimaParcela'])
    valorPriParS = str(textoSimulador['valorPrimeiraParcela']).replace(".",",")
    valorPriParEncargosS = str(textoSimulador['valorPrimeiraParcelaComEncargos']).replace(".",",")
    valorAvaliacaoS = str(textoSimulador['valorImóvel']).replace(".",",")

    #Clear Box
    tbSValorBruto.delete(0, 'end')
    tbSValorLiquido.delete(0, 'end')
    tbSAmortizacao.delete(0, 'end')
    tbSIndice.delete(0, 'end')
    tbSPrazo.delete(0, 'end')
    tbSParcelas.delete(0, 'end')
    tbSDataDesem.delete(0, 'end')
    tbSDataPriParcela.delete(0, 'end')
    tbSDataUltimaParcela.delete(0, 'end')
    tbSValorPriPar.delete(0, 'end')
    tbSValorPriEncargosPar.delete(0, 'end')
    tbSValorAvaliacao.delete(0, 'end')

    #Inserir dados do Contrato
    tbSValorBruto.insert(END, valorBrutoS)
    tbSValorLiquido.insert(END, valorLiquidoS)
    tbSAmortizacao.insert(END, valorAmortizacaoS)
    tbSIndice.insert(END, valorIndiceS)
    tbSPrazo.insert(END, prazoS)
    tbSParcelas.insert(END, parcelasContratoS)
    tbSDataDesem.insert(END, dataDesembolcoS)
    tbSDataPriParcela.insert(END, dataPriParS)
    tbSDataUltimaParcela.insert(END, dataUltimaParS)
    tbSValorPriPar.insert(END, valorPriParS)
    tbSValorPriEncargosPar.insert(END, valorPriParEncargosS)
    tbSValorAvaliacao.insert(END, valorAvaliacaoS)

#Ler o Laudo
def infosLaudo(pathLaudo):
    textoLaudo = lerLaudo(pathLaudo)

    #Separar Key-Value
    
    #CEP
    #Tentar valores via API pelo CEP
    try:
        
        valorCEPL = str(textoLaudo['CEP '])
        try:
            address = get_address_from_cep(valorCEPL, webservice=WebService.APICEP)
        except:
            cep = str(textoLaudo['CEP '])
            cep = ''.join([d for d in cep if d.isdigit()])
            if len(cep) == 8:
                address = get_address_from_cep(valorCEPL, webservice=WebService.APICEP)
        
        #Pegar Valores API
        valorEnderecoL = str(address['logradouro'])
        valorBairroL = str(address['bairro'])
        valorCidadeL = str(address['cidade'])
        valorUF = str(address['uf'])
    #Extrair valores do Laudo caso API Falhe
    except:
        valorEnderecoL = str(textoLaudo['Endereço: '])
        valorBairroL = str(textoLaudo['Bairro: '])
        valorCidadeL = str(textoLaudo['Cidade: '])
        valorUF = str(textoLaudo['UF: '])
        
    valorNumeroL = str(textoLaudo['Número: '])
    valorComplementoL = str(textoLaudo['Complemento: '])
    valorAvaliacaoL = str(textoLaudo['Valor de Mercado: '])
    matriculaImovelL = str(textoLaudo['Matrícula: '])
    cartorioImovelL = str(textoLaudo['Cartório'])

    #Tratar Números
    valorAvaliacaoL = ''.join([d for d in valorAvaliacaoL if d.isdigit() or d == ','])
    matriculaImovelL = ''.join([d for d in matriculaImovelL if d.isdigit()])

    #Clear Box
    tbLEndereco.delete(0, 'end')
    tbLNumeroEndereco.delete(0, 'end')
    tbLComplemento.delete(0, 'end')
    tbLCEP.delete(0, 'end')
    tbLBairro.delete(0, 'end')
    tbLCidade.delete(0, 'end')
    tbLUF.delete(0, 'end')
    tbLValorAvaliacao.delete(0, 'end')
    tbLMatricula.delete(0, 'end')
    tbLCartorio.delete(0, 'end')

    #Inserir dados do Contrato
    tbLEndereco.insert(END, valorEnderecoL)
    tbLNumeroEndereco.insert(END, valorNumeroL)
    tbLComplemento.insert(END, valorComplementoL)
    tbLCEP.insert(END, valorCEPL)
    tbLBairro.insert(END, valorBairroL)
    tbLCidade.insert(END, valorCidadeL)
    tbLUF.insert(END, valorUF)
    tbLValorAvaliacao.insert(END, valorAvaliacaoL)
    tbLMatricula.insert(END, matriculaImovelL)
    tbLCartorio.insert(END, cartorioImovelL)

#Função Validar Itens
def validarEntry():
    
    #Var Globais Contrato
    global tbCValorBruto
    global tbCValorLiquido
    global tbCAmortizacao
    global tbCIndice
    global tbCPrazo
    global tbCParcelas
    global tbCDataDesem
    global tbCDataPriParcela
    global tbCDataUltimaParcela
    global tbCValorPriPar
    global tbCValorPriEncargosPar
    global tbCValorAvaliacao
    global tbCMatricula
    global tbCCartorio
    global tbCNumParticipantes
    global tbCDataContrato

    #Var Globais Simulador
    global tbSValorBruto
    global tbSValorLiquido
    global tbSAmortizacao
    global tbSIndice
    global tbSPrazo
    global tbSParcelas
    global tbSDataDesem
    global tbSDataPriParcela
    global tbSDataUltimaParcela
    global tbSValorPriPar
    global tbSValorPriEncargosPar
    global tbSValorAvaliacao

    #Var Globais Laudo
    global tbLEndereco
    global tbLNumeroEndereco
    global tbLComplemento
    global tbLCEP
    global tbLBairro
    global tbLCidade
    global tbLUF
    global tbLUnidade
    global tbLBloco
    global tbLValorAvaliacao
    global tbLMatricula
    global tbLCartorio
    global tbLEndereco
    global tbLNumeroEndereco
    global tbLComplemento
    global tbLBairro
    global tbLCEP
    global tbLCidade
    global tbLUF
    global tbLMatricula
    global tbLUnidade
    global tbLBloco
    

    #Definção de Lista de Contrato
    listaContrato = [tbCValorBruto, tbCValorLiquido, tbCAmortizacao, tbCIndice, tbCPrazo, tbCParcelas, tbCDataDesem, tbCDataPriParcela,
                    tbCDataUltimaParcela, tbCValorPriPar, tbCValorPriEncargosPar, tbCValorAvaliacao, tbCMatricula, tbCCartorio
                    ]

    #Definção de Lista do Simulador
    listaSimulador = [tbSValorBruto, tbSValorLiquido, tbSAmortizacao, tbSIndice, tbSPrazo, tbSParcelas, tbSDataDesem, tbSDataPriParcela,
                        tbSDataUltimaParcela, tbSValorPriPar, tbSValorPriEncargosPar, tbSValorAvaliacao, tbLMatricula, tbLCartorio
                    ]
    
    tamanhoLista = len(listaContrato)

    for i in range(tamanhoLista):
        num = i + 1
        varContrato = listaContrato[i].get() 
        varSimulador = listaSimulador[i].get()
        
        #Corrigir Espaço Indesejado
        if i<13:
            varContrato = varContrato.replace(" ", "")
            varSimulador = varSimulador.replace(" ", "")
        
        #Transformar Valores Númericos
        if i == 0 or i == 1 or i == 9 or i == 10 or i == 11:
            varContrato = round(float(varContrato.replace(",",".")),2)
            varSimulador = round(float(varSimulador.replace(",",".")),2)

        if i == 2 or i == 3:
            varContrato = varContrato.lower()
            varSimulador = varSimulador.lower()

        #Comparação de Valores
        if varContrato == varSimulador:
            botaoUm = "VERDADEIRO"
            color = 'green'
            fontColor = 'white'
            
        else:
            botaoUm = "FALSO"
            color = 'red'
            fontColor = 'white'
    
        # #Criar Botão de Validação
        validValorBruto = tk.Button(
             canvas,
             text = botaoUm,
             bg=color,
             fg=fontColor
             )
        if i<11:
            validValorBruto.grid(row=8+num, column=2)
        else:
            validValorBruto.grid(row=12+num, column=2)

#Função UploadFiles
def uploadFiles():
    global contrato
    global calculoFluxo
    pb1 = Progressbar(
        canvas, 
        orient=HORIZONTAL, 
        length=300, 
        mode='determinate'
        )
    pb1.grid(row=4, columnspan=3, pady=20)
    for i in range(5):
        canvas.update_idletasks()
        pb1['value'] += 20
        time.sleep(1)
    pb1.destroy()
    msg = savePaths()
    if msg == 0:
        textoUpload = "Todos os arquivos foram anexados corretamente. Siga com a validação dos dados apresentados abaixo."
        color = 'green'
    elif msg ==1:
        textoUpload = "Favor selecionar todos os arquivos antes de seguir para os próximos passos."
        color = 'red'
    Label(canvas, text=textoUpload, foreground=color).grid(row=4, columnspan=3, pady=10)
    infosContrato(contrato)
    infosSimulador(simulador)
    infosLaudo(laudo)

#Pegar Valores Contrato
def getValuesContrato():
    infosContrato = {'tabela': tbCAmortizacao.get(), 'indice': tbCIndice.get(), 'dataContrato': tbCDataContrato.get(), 'taxaAoAno': textBoxTaxaJuros.get(),
                    'Matrícula':tbCMatricula.get(), 'Quantidade': tbCNumParticipantes.get(), 'operação':tipoOps,'CCI':valorCCI}
    return infosContrato

#Pegar valores do Imóvel (Laudo)
def getValuesImovel():
    try:
        global tbLEndereco
        global tbLNumeroEndereco
        global tbLComplemento
        global tbLBairro
        global tbLCEP
        global tbLCidade
        global tbLUF
        global tbLMatricula
        global tbLUnidade
        global tbLBloco
    except:
        pass

    infosImovel = {'enderecoImovel':tbLEndereco.get(), 'numeroImovel':tbLNumeroEndereco.get(), 'complementoImovel':tbLComplemento.get(), 'bairroImovel':tbLBairro.get(), 
    'cepImovel': tbLCEP.get(), 'cidadeImovel':tbLCidade.get(), 'estadoImovel':tbLUF.get(), 'unidadeImovel':tbLUnidade.get(), 'blocoImovel':tbLBloco.get(), 'valorImovel': tbLValorAvaliacao.get()
                    }
    return infosImovel

#Função Adicionar Campos de Participantes
def camposParcipantes():
    try:
        global textoParticipantes
        global nParticipante
        global dadosC 
        global dadosI
        global listaFinal

        #Variáveis Globais
        try:
            global tbCNomeP1
            global tbCCpfP1
            global tbCDataNascimentoP1
            global tbCCidadeP1
            global tbCEnderecodP1
            global tbCNumeroP1
            global tbCComplementoP1
            global tbCBairroP1
            global tbCUFP1
            global tbCCEPP1
            global tbCTelefoneP1
            global tbCEmailP1
            global tbCparticipacaoP1
            global tbCNomeP2
            global tbCCpfP2
            global tbCDataNascimentoP2
            global tbCCidadeP2
            global tbCEnderecodP2
            global tbCNumeroP2
            global tbCComplementoP2
            global tbCBairroP2
            global tbCUFP2
            global tbCCEPP2
            global tbCTelefoneP2
            global tbCEmailP2
            global tbCparticipacaoP2
            global tbCNomeP3
            global tbCCpfP3
            global tbCDataNascimentoP3
            global tbCCidadeP3
            global tbCEnderecodP3
            global tbCNumeroP3
            global tbCComplementoP3
            global tbCBairroP3
            global tbCUFP3
            global tbCCEPP3
            global tbCTelefoneP3
            global tbCEmailP3
            global tbCparticipacaoP3
            global tbCNomeP4
            global tbCCpfP4
            global tbCDataNascimentoP4
            global tbCCidadeP4
            global tbCEnderecodP4
            global tbCNumeroP4
            global tbCComplementoP4
            global tbCBairroP4
            global tbCUFP4
            global tbCCEPP4
            global tbCTelefoneP4
            global tbCEmailP4
            global tbCparticipacaoP4
            global tbCNomeP5
            global tbCCpfP5
            global tbCDataNascimentoP5
            global tbCCidadeP5
            global tbCEnderecodP5
            global tbCNumeroP5
            global tbCComplementoP5
            global tbCBairroP5
            global tbCUFP5
            global tbCCEPP5
            global tbCTelefoneP5
            global tbCEmailP5
            global tbCparticipacaoP5
            global tbLEndereco
            global tbLNumeroEndereco
            global tbLComplemento
            global tbLBairro
            global tbLCEP
            global tbLCidade
            global tbLUF
            global tbLMatricula
            global tbLUnidade
            global tbLBloco
        except:
            pass

        #Participante 1
        nParticipante = int(tbCNumParticipantes.get())
        if nParticipante > 5:
            tk.messagebox.showwarning(title='ATENÇÃO!', message='Atualmente o programa está configurado\npara receber até 5 participantes.\nFavor entrar em contato com a WAQ Corporations.')
        
        i=1
        j=0

        #Adicionar Campos de Participantes

        if 1 <= nParticipante:
            #Participante 1:
            try:
                #Definir Número de Participantes
                participante1 = Label(
                    canvas, 
                    text=f'Participante {i}',
                    font=("bold",15),
                    )
                participante1.grid(row=30+j, column=0,pady=20)

                    #Nome Participante
                try:
                    botaoNomeP1C = Label(
                        canvas, 
                        text='Nome Completo'
                        )
                    botaoNomeP1C.grid(row=31+j, column=0, padx=20)
                    tbCNomeP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCNomeP1.grid(row=31+j, column=1, pady=0)
                except:
                    pass

                #CPF Participante 1
                try:
                    botaoCpfP1C = Label(
                        canvas, 
                        text='CPF'
                        )
                    botaoCpfP1C.grid(row=31+j, column=2, padx=20)
                    tbCCpfP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCpfP1.grid(row=31+j, column=3, pady=0)
                except:
                    pass

                #Data Nascimento Participante 1
                try:
                    botaoDataNascimentoP1C = Label(
                        canvas, 
                        text='Data Nascimento'
                        )
                    botaoDataNascimentoP1C.grid(row=31+j, column=4, padx=20)
                    tbCDataNascimentoP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCDataNascimentoP1.grid(row=31+j, column=5, pady=0)
                except:
                    pass

                #Cidade Participante 1
                try:
                    botaoCidadeP1C = Label(
                        canvas, 
                        text='Cidade'
                        )
                    botaoCidadeP1C.grid(row=31+j, column=6, padx=20)
                    tbCCidadeP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCidadeP1.grid(row=31+j, column=7, pady=0)
                except:
                    pass

                #Endereço Participante 1
                try:
                    botaoEnderecoP1C = Label(
                        canvas, 
                        text='Endereço'
                        )
                    botaoEnderecoP1C.grid(row=31+j, column=8, padx=20)
                    tbCEnderecodP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCEnderecodP1.grid(row=31+j, column=9, pady=0)
                except:
                    pass

                #Número Participante 1
                try:
                    botaoNumeroP1C = Label(
                        canvas, 
                        text='Numero'
                        )
                    botaoNumeroP1C.grid(row=32+j, column=0, padx=20)
                    tbCNumeroP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCNumeroP1.grid(row=32+j, column=1, pady=5)
                except:
                    pass

                #Complemento Participante 1
                try:
                    botaoComplementoP1C = Label(
                        canvas, 
                        text='Complemento'
                        )
                    botaoComplementoP1C.grid(row=32+j, column=2, padx=20)
                    tbCComplementoP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCComplementoP1.grid(row=32+j, column=3, pady=0)
                except:
                    pass

                #Bairro Participante 1
                try:
                    botaoBairroP1C = Label(
                        canvas, 
                        text='Bairro'
                        )
                    botaoBairroP1C.grid(row=32+j, column=4, padx=20)
                    tbCBairroP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCBairroP1.grid(row=32+j, column=5, pady=0)
                except:
                    pass

                #UF Participante 1
                try:
                    botaoUFP1C = Label(
                        canvas, 
                        text='UF     '
                        )
                    botaoUFP1C.grid(row=32+j, column=6, padx=20)
                    tbCUFP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCUFP1.grid(row=32+j, column=7, pady=0)
                except:
                    pass

                #CEP Participante 1
                try:
                    botaoCEPP1C = Label(
                        canvas, 
                        text='CEP'
                        )
                    botaoCEPP1C.grid(row=32+j, column=8, padx=20)
                    tbCCEPP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCEPP1.grid(row=32+j, column=9, pady=0)
                except:
                    pass

                #Telefone Participante 1
                try:
                    botaoTelefoneP1C = Label(
                        canvas, 
                        text='Telefone'
                        )
                    botaoTelefoneP1C.grid(row=33+j, column=0, padx=20)
                    tbCTelefoneP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCTelefoneP1.grid(row=33+j, column=1, pady=0)
                except:
                    pass

                #Email Participante 1
                try:
                    botaoEmailP1C = Label(
                        canvas, 
                        text='Email'
                        )
                    botaoEmailP1C.grid(row=33+j, column=2, padx=20)
                    tbCEmailP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCEmailP1.grid(row=33+j, column=3, pady=0)
                except:
                    pass

                #Tipo de Operação
                try:
                    tipoOpP1C = Label(
                        canvas, 
                        text='Tipo Operação',
                        )
                    tipoOpP1C.grid(row=33+j, column=4)
                    variableOps1 = StringVar(canvas)
                    tbCTipoOps1 = OptionMenu(canvas, variableOps1, "", "PF", "PJ")
                    tbCTipoOps1.grid(row=33+j, column=5)
                except:
                    pass

                #Sexo Operação
                try:
                    sexoP1C = Label(
                        canvas, 
                        text='Sexo'
                        )
                    sexoP1C.grid(row=33+j, column=6)
                    variableSexo1 = StringVar(canvas)
                    canvasexoP1 = OptionMenu(canvas, variableSexo1, "", "F", "M")
                    canvasexoP1.grid(row=33+j, column=7)
                except:
                    pass

                #Relação
                try:
                    relacaoP1C = Label(
                        canvas, 
                        text='Relação'
                        )
                    relacaoP1C.grid(row=33+j, column=8)
                    variableRelacao1 = StringVar(canvas)
                    wRelacaoP1 = OptionMenu(canvas, variableRelacao1, "", "Titular", "Cônjuge", "Procurador" ,"Pai", "Mãe", "Irmão")
                    wRelacaoP1.grid(row=33+j, column=9)
                except:
                    pass

                #Participação
                try:
                    participacaoP1C = Label(
                        canvas, 
                        text='Participação'
                        )
                    participacaoP1C.grid(row=34+j, column=0, padx=20)
                    tbCparticipacaoP1 = Entry(
                        canvas,
                        width=20
                        )
                    tbCparticipacaoP1.grid(row=34+j, column=1, pady=5)
                except:
                    pass
            except:
                print('Não foi possível inserir o participante 1')

            #### Adicionar Mais um ####
            
            i+= 1
            
            if i>1:
                j=5+j
        
        if 2 <= nParticipante:
            #Participante 2:
            try:
                #Definir Número de Participantes
                P2C = Label(
                    canvas, 
                    text=f'Participante {i}',
                    font=("bold",15),
                    )
                P2C.grid(row=30+j, column=0,pady=20)

                    #Nome Participante
                try:
                    botaoNomeP2C = Label(
                        canvas, 
                        text='Nome Completo'
                        )
                    botaoNomeP2C.grid(row=31+j, column=0, padx=20)
                    tbCNomeP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCNomeP2.grid(row=31+j, column=1, pady=0)
                except:
                    pass

                #CPF Participante 2
                try:
                    botaoCpfP2C = Label(
                        canvas, 
                        text='CPF'
                        )
                    botaoCpfP2C.grid(row=31+j, column=2, padx=20)
                    tbCCpfP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCpfP2.grid(row=31+j, column=3, pady=0)
                except:
                    pass

                #Data Nascimento Participante 2
                try:
                    botaoDataNascimentoP2C = Label(
                        canvas, 
                        text='Data Nascimento'
                        )
                    botaoDataNascimentoP2C.grid(row=31+j, column=4, padx=20)
                    tbCDataNascimentoP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCDataNascimentoP2.grid(row=31+j, column=5, pady=0)
                except:
                    pass

                #Cidade Participante 2
                try:
                    botaoCidadeP2C = Label(
                        canvas, 
                        text='Cidade'
                        )
                    botaoCidadeP2C.grid(row=31+j, column=6, padx=20)
                    tbCCidadeP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCidadeP2.grid(row=31+j, column=7, pady=0)
                except:
                    pass

                #Endereço Participante 2
                try:
                    botaoEnderecoP2C = Label(
                        canvas, 
                        text='Endereço'
                        )
                    botaoEnderecoP2C.grid(row=31+j, column=8, padx=20)
                    tbCEnderecodP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCEnderecodP2.grid(row=31+j, column=9, pady=0)
                except:
                    pass

                #Número Participante 2
                try:
                    botaoNumeroP2C = Label(
                        canvas, 
                        text='Numero'
                        )
                    botaoNumeroP2C.grid(row=32+j, column=0, padx=20)
                    tbCNumeroP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCNumeroP2.grid(row=32+j, column=1, pady=5)
                except:
                    pass

                #Complemento Participante 2
                try:
                    botaoComplementoP2C = Label(
                        canvas, 
                        text='Complemento'
                        )
                    botaoComplementoP2C.grid(row=32+j, column=2, padx=20)
                    tbCComplementoP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCComplementoP2.grid(row=32+j, column=3, pady=0)
                except:
                    pass

                #Bairro Participante 2
                try:
                    botaoBairroP2C = Label(
                        canvas, 
                        text='Bairro'
                        )
                    botaoBairroP2C.grid(row=32+j, column=4, padx=20)
                    tbCBairroP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCBairroP2.grid(row=32+j, column=5, pady=0)
                except:
                    pass

                #UF Participante 2
                try:
                    botaoUFP2C = Label(
                        canvas, 
                        text='UF     '
                        )
                    botaoUFP2C.grid(row=32+j, column=6, padx=20)
                    tbCUFP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCUFP2.grid(row=32+j, column=7, pady=0)
                except:
                    pass

                #CEP Participante 2
                try:
                    botaoCEPP2C = Label(
                        canvas, 
                        text='CEP'
                        )
                    botaoCEPP2C.grid(row=32+j, column=8, padx=20)
                    tbCCEPP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCEPP2.grid(row=32+j, column=9, pady=0)
                except:
                    pass

                #Telefone Participante 2
                try:
                    botaoTelefoneP2C = Label(
                        canvas, 
                        text='Telefone'
                        )
                    botaoTelefoneP2C.grid(row=33+j, column=0, padx=20)
                    tbCTelefoneP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCTelefoneP2.grid(row=33+j, column=1, pady=0)
                except:
                    pass

                #Email Participante 2
                try:
                    botaoEmailP2C = Label(
                        canvas, 
                        text='Email'
                        )
                    botaoEmailP2C.grid(row=33+j, column=2, padx=20)
                    tbCEmailP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCEmailP2.grid(row=33+j, column=3, pady=0)
                except:
                    pass

                #Tipo de Operação
                try:
                    tipoOpP2C = Label(
                        canvas, 
                        text='Tipo Operação',
                        )
                    tipoOpP2C.grid(row=33+j, column=4)
                    variableOps2 = StringVar(canvas)
                    tbCTipoOps2 = OptionMenu(canvas, variableOps2, "", "PF", "PJ")
                    tbCTipoOps2.grid(row=33+j, column=5)
                except:
                    pass

                #Sexo Operação
                try:
                    sexoP2C = Label(
                        canvas, 
                        text='Sexo'
                        )
                    sexoP2C.grid(row=33+j, column=6)
                    variableSexo2 = StringVar(canvas)
                    canvasexoP2 = OptionMenu(canvas, variableSexo2, "", "F", "M")
                    canvasexoP2.grid(row=33+j, column=7)
                except:
                    pass

                #Relação
                try:
                    relacaoP2C = Label(
                        canvas, 
                        text='Relação'
                        )
                    relacaoP2C.grid(row=33+j, column=8)
                    variableRelacao2 = StringVar(canvas)
                    wRelacaoP2 = OptionMenu(canvas, variableRelacao2, "", "Titular", "Cônjuge", "Procurador" ,"Pai", "Mãe", "Irmão")
                    wRelacaoP2.grid(row=33+j, column=9)
                except:
                    pass

                #Participação
                try:
                    participacaoP2C = Label(
                        canvas, 
                        text='Participação'
                        )
                    participacaoP2C.grid(row=34+j, column=0, padx=20)
                    tbCparticipacaoP2 = Entry(
                        canvas,
                        width=20
                        )
                    tbCparticipacaoP2.grid(row=34+j, column=1, pady=5)
                except:
                    pass
            except:
                print('Não foi possível inserir o participante 1')
            
            #Adicionar Mais um
            i+= 1

            #Ajuste de Coluna    
            if i>1:
                j=5+j
        
        if 3 <= nParticipante:
            #Participante 3:
            try:
                #Definir Número de Participantes
                P3C = Label(
                    canvas, 
                    text=f'Participante {i}',
                    font=("bold",15),
                    )
                P3C.grid(row=30+j, column=0,pady=20)

                    #Nome Participante
                try:
                    botaoNomeP3C = Label(
                        canvas, 
                        text='Nome Completo'
                        )
                    botaoNomeP3C.grid(row=31+j, column=0, padx=20)
                    tbCNomeP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCNomeP3.grid(row=31+j, column=1, pady=0)
                except:
                    pass

                #CPF Participante 3
                try:
                    botaoCpfP3C = Label(
                        canvas, 
                        text='CPF'
                        )
                    botaoCpfP3C.grid(row=31+j, column=2, padx=20)
                    tbCCpfP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCpfP3.grid(row=31+j, column=3, pady=0)
                except:
                    pass

                #Data Nascimento Participante 3
                try:
                    botaoDataNascimentoP3C = Label(
                        canvas, 
                        text='Data Nascimento'
                        )
                    botaoDataNascimentoP3C.grid(row=31+j, column=4, padx=20)
                    tbCDataNascimentoP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCDataNascimentoP3.grid(row=31+j, column=5, pady=0)
                except:
                    pass

                #Cidade Participante 3
                try:
                    botaoCidadeP3C = Label(
                        canvas, 
                        text='Cidade'
                        )
                    botaoCidadeP3C.grid(row=31+j, column=6, padx=20)
                    tbCCidadeP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCidadeP3.grid(row=31+j, column=7, pady=0)
                except:
                    pass

                #Endereço Participante 3
                try:
                    botaoEnderecoP3C = Label(
                        canvas, 
                        text='Endereço'
                        )
                    botaoEnderecoP3C.grid(row=31+j, column=8, padx=20)
                    tbCEnderecodP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCEnderecodP3.grid(row=31+j, column=9, pady=0)
                except:
                    pass

                #Número Participante 3
                try:
                    botaoNumeroP3C = Label(
                        canvas, 
                        text='Numero'
                        )
                    botaoNumeroP3C.grid(row=32+j, column=0, padx=20)
                    tbCNumeroP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCNumeroP3.grid(row=32+j, column=1, pady=5)
                except:
                    pass

                #Complemento Participante 3
                try:
                    botaoComplementoP3C = Label(
                        canvas, 
                        text='Complemento'
                        )
                    botaoComplementoP3C.grid(row=32+j, column=2, padx=20)
                    tbCComplementoP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCComplementoP3.grid(row=32+j, column=3, pady=0)
                except:
                    pass

                #Bairro Participante 3
                try:
                    botaoBairroP3C = Label(
                        canvas, 
                        text='Bairro'
                        )
                    botaoBairroP3C.grid(row=32+j, column=4, padx=20)
                    tbCBairroP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCBairroP3.grid(row=32+j, column=5, pady=0)
                except:
                    pass

                #UF Participante 3
                try:
                    botaoUFP3C = Label(
                        canvas, 
                        text='UF     '
                        )
                    botaoUFP3C.grid(row=32+j, column=6, padx=20)
                    tbCUFP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCUFP3.grid(row=32+j, column=7, pady=0)
                except:
                    pass

                #CEP Participante 3
                try:
                    botaoCEPP3C = Label(
                        canvas, 
                        text='CEP'
                        )
                    botaoCEPP3C.grid(row=32+j, column=8, padx=20)
                    tbCCEPP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCEPP3.grid(row=32+j, column=9, pady=0)
                except:
                    pass

                #Telefone Participante 3
                try:
                    botaoTelefoneP3C = Label(
                        canvas, 
                        text='Telefone'
                        )
                    botaoTelefoneP3C.grid(row=33+j, column=0, padx=20)
                    tbCTelefoneP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCTelefoneP3.grid(row=33+j, column=1, pady=0)
                except:
                    pass

                #Email Participante 3
                try:
                    botaoEmailP3C = Label(
                        canvas, 
                        text='Email'
                        )
                    botaoEmailP3C.grid(row=33+j, column=2, padx=20)
                    tbCEmailP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCEmailP3.grid(row=33+j, column=3, pady=0)
                except:
                    pass

                #Tipo de Operação
                try:
                    tipoOpP3C = Label(
                        canvas, 
                        text='Tipo Operação',
                        )
                    tipoOpP3C.grid(row=33+j, column=4)
                    variableOps3 = StringVar(canvas)
                    tbCTipoOps3 = OptionMenu(canvas, variableOps3, "", "PF", "PJ")
                    tbCTipoOps3.grid(row=33+j, column=5)
                except:
                    pass

                #Sexo Operação
                try:
                    sexoP3C = Label(
                        canvas, 
                        text='Sexo'
                        )
                    sexoP3C.grid(row=33+j, column=6)
                    variableSexo3 = StringVar(canvas)
                    canvasexoP3 = OptionMenu(canvas, variableSexo3, "", "F", "M")
                    canvasexoP3.grid(row=33+j, column=7)
                except:
                    pass

                #Relação
                try:
                    relacaoP3C = Label(
                        canvas, 
                        text='Relação'
                        )
                    relacaoP3C.grid(row=33+j, column=8)
                    variableRelacao3 = StringVar(canvas)
                    wRelacaoP3 = OptionMenu(canvas, variableRelacao3, "", "Titular", "Cônjuge", "Procurador" ,"Pai", "Mãe", "Irmão")
                    wRelacaoP3.grid(row=33+j, column=9)
                except:
                    pass

                #Participação
                try:
                    participacaoP3C = Label(
                        canvas, 
                        text='Participação'
                        )
                    participacaoP3C.grid(row=34+j, column=0, padx=20)
                    tbCparticipacaoP3 = Entry(
                        canvas,
                        width=20
                        )
                    tbCparticipacaoP3.grid(row=34+j, column=1, pady=5)
                except:
                    pass
            except:
                print('Não foi possível inserir o participante 1')
            
            #Adicionar Mais um
            i+= 1

            #Ajuste de Coluna    
            if i>1:
                j=5+j

        if 4 <= nParticipante:
            #Participante 4:
            try:
                #Definir Número de Participantes
                P4C = Label(
                    canvas, 
                    text=f'Participante {i}',
                    font=("bold",15),
                    )
                P4C.grid(row=30+j, column=0,pady=20)

                    #Nome Participante
                try:
                    botaoNomeP4C = Label(
                        canvas, 
                        text='Nome Completo'
                        )
                    botaoNomeP4C.grid(row=31+j, column=0, padx=20)
                    tbCNomeP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCNomeP4.grid(row=31+j, column=1, pady=0)
                except:
                    pass

                #CPF Participante 4
                try:
                    botaoCpfP4C = Label(
                        canvas, 
                        text='CPF'
                        )
                    botaoCpfP4C.grid(row=31+j, column=2, padx=20)
                    tbCCpfP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCpfP4.grid(row=31+j, column=3, pady=0)
                except:
                    pass

                #Data Nascimento Participante 4
                try:
                    botaoDataNascimentoP4C = Label(
                        canvas, 
                        text='Data Nascimento'
                        )
                    botaoDataNascimentoP4C.grid(row=31+j, column=4, padx=20)
                    tbCDataNascimentoP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCDataNascimentoP4.grid(row=31+j, column=5, pady=0)
                except:
                    pass

                #Cidade Participante 4
                try:
                    botaoCidadeP4C = Label(
                        canvas, 
                        text='Cidade'
                        )
                    botaoCidadeP4C.grid(row=31+j, column=6, padx=20)
                    tbCCidadeP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCidadeP4.grid(row=31+j, column=7, pady=0)
                except:
                    pass

                #Endereço Participante 4
                try:
                    botaoEnderecoP4C = Label(
                        canvas, 
                        text='Endereço'
                        )
                    botaoEnderecoP4C.grid(row=31+j, column=8, padx=20)
                    tbCEnderecodP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCEnderecodP4.grid(row=31+j, column=9, pady=0)
                except:
                    pass

                #Número Participante 4
                try:
                    botaoNumeroP4C = Label(
                        canvas, 
                        text='Numero'
                        )
                    botaoNumeroP4C.grid(row=32+j, column=0, padx=20)
                    tbCNumeroP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCNumeroP4.grid(row=32+j, column=1, pady=5)
                except:
                    pass

                #Complemento Participante 4
                try:
                    botaoComplementoP4C = Label(
                        canvas, 
                        text='Complemento'
                        )
                    botaoComplementoP4C.grid(row=32+j, column=2, padx=20)
                    tbCComplementoP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCComplementoP4.grid(row=32+j, column=3, pady=0)
                except:
                    pass

                #Bairro Participante 4
                try:
                    botaoBairroP4C = Label(
                        canvas, 
                        text='Bairro'
                        )
                    botaoBairroP4C.grid(row=32+j, column=4, padx=20)
                    tbCBairroP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCBairroP4.grid(row=32+j, column=5, pady=0)
                except:
                    pass

                #UF Participante 4
                try:
                    botaoUFP4C = Label(
                        canvas, 
                        text='UF     '
                        )
                    botaoUFP4C.grid(row=32+j, column=6, padx=20)
                    tbCUFP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCUFP4.grid(row=32+j, column=7, pady=0)
                except:
                    pass

                #CEP Participante 4
                try:
                    botaoCEPP4C = Label(
                        canvas, 
                        text='CEP'
                        )
                    botaoCEPP4C.grid(row=32+j, column=8, padx=20)
                    tbCCEPP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCEPP4.grid(row=32+j, column=9, pady=0)
                except:
                    pass

                #Telefone Participante 4
                try:
                    botaoTelefoneP4C = Label(
                        canvas, 
                        text='Telefone'
                        )
                    botaoTelefoneP4C.grid(row=33+j, column=0, padx=20)
                    tbCTelefoneP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCTelefoneP4.grid(row=33+j, column=1, pady=0)
                except:
                    pass

                #Email Participante 4
                try:
                    botaoEmailP4C = Label(
                        canvas, 
                        text='Email'
                        )
                    botaoEmailP4C.grid(row=33+j, column=2, padx=20)
                    tbCEmailP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCEmailP4.grid(row=33+j, column=3, pady=0)
                except:
                    pass

                #Tipo de Operação
                try:
                    tipoOpP4C = Label(
                        canvas, 
                        text='Tipo Operação',
                        )
                    tipoOpP4C.grid(row=33+j, column=4)
                    variableOps4 = StringVar(canvas)
                    tbCTipoOps4 = OptionMenu(canvas, variableOps4, "", "PF", "PJ")
                    tbCTipoOps4.grid(row=33+j, column=5)
                except:
                    pass

                #Sexo Operação
                try:
                    sexoP4C = Label(
                        canvas, 
                        text='Sexo'
                        )
                    sexoP4C.grid(row=33+j, column=6)
                    variableSexo4 = StringVar(canvas)
                    canvasexoP4 = OptionMenu(canvas, variableSexo4, "", "F", "M")
                    canvasexoP4.grid(row=33+j, column=7)
                except:
                    pass

                #Relação
                try:
                    relacaoP4C = Label(
                        canvas, 
                        text='Relação'
                        )
                    relacaoP4C.grid(row=33+j, column=8)
                    variableRelacao4 = StringVar(canvas)
                    wRelacaoP4 = OptionMenu(canvas, variableRelacao4, "", "Titular", "Cônjuge", "Procurador" ,"Pai", "Mãe", "Irmão")
                    wRelacaoP4.grid(row=33+j, column=9)
                except:
                    pass

                #Participação
                try:
                    participacaoP4C = Label(
                        canvas, 
                        text='Participação'
                        )
                    participacaoP4C.grid(row=34+j, column=0, padx=20)
                    tbCparticipacaoP4 = Entry(
                        canvas,
                        width=20
                        )
                    tbCparticipacaoP4.grid(row=34+j, column=1, pady=5)
                except:
                    pass
            except:
                print('Não foi possível inserir o participante 1')
            
            #Adicionar Mais um
            i+= 1

            #Ajuste de Coluna    
            if i>1:
                j=5+j             

        if 5 <= nParticipante:
            #Participante 4:
            try:
                #Definir Número de Participantes
                P5C = Label(
                    canvas, 
                    text=f'Participante {i}',
                    font=("bold",15),
                    )
                P5C.grid(row=30+j, column=0,pady=20)

                    #Nome Participante
                try:
                    botaoNomeP5C = Label(
                        canvas, 
                        text='Nome Completo'
                        )
                    botaoNomeP5C.grid(row=31+j, column=0, padx=20)
                    tbCNomeP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCNomeP5.grid(row=31+j, column=1, pady=0)
                except:
                    pass

                #CPF Participante 5
                try:
                    botaoCpfP5C = Label(
                        canvas, 
                        text='CPF'
                        )
                    botaoCpfP5C.grid(row=31+j, column=2, padx=20)
                    tbCCpfP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCpfP5.grid(row=31+j, column=3, pady=0)
                except:
                    pass

                #Data Nascimento Participante 5
                try:
                    botaoDataNascimentoP5C = Label(
                        canvas, 
                        text='Data Nascimento'
                        )
                    botaoDataNascimentoP5C.grid(row=31+j, column=4, padx=20)
                    tbCDataNascimentoP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCDataNascimentoP5.grid(row=31+j, column=5, pady=0)
                except:
                    pass

                #Cidade Participante 5
                try:
                    botaoCidadeP5C = Label(
                        canvas, 
                        text='Cidade'
                        )
                    botaoCidadeP5C.grid(row=31+j, column=6, padx=20)
                    tbCCidadeP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCidadeP5.grid(row=31+j, column=7, pady=0)
                except:
                    pass

                #Endereço Participante 5
                try:
                    botaoEnderecoP5C = Label(
                        canvas, 
                        text='Endereço'
                        )
                    botaoEnderecoP5C.grid(row=31+j, column=8, padx=20)
                    tbCEnderecodP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCEnderecodP5.grid(row=31+j, column=9, pady=0)
                except:
                    pass

                #Número Participante 5
                try:
                    botaoNumeroP5C = Label(
                        canvas, 
                        text='Numero'
                        )
                    botaoNumeroP5C.grid(row=32+j, column=0, padx=20)
                    tbCNumeroP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCNumeroP5.grid(row=32+j, column=1, pady=5)
                except:
                    pass

                #Complemento Participante 5
                try:
                    botaoComplementoP5C = Label(
                        canvas, 
                        text='Complemento'
                        )
                    botaoComplementoP5C.grid(row=32+j, column=2, padx=20)
                    tbCComplementoP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCComplementoP5.grid(row=32+j, column=3, pady=0)
                except:
                    pass

                #Bairro Participante 5
                try:
                    botaoBairroP5C = Label(
                        canvas, 
                        text='Bairro'
                        )
                    botaoBairroP5C.grid(row=32+j, column=4, padx=20)
                    tbCBairroP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCBairroP5.grid(row=32+j, column=5, pady=0)
                except:
                    pass

                #UF Participante 5
                try:
                    botaoUFP5C = Label(
                        canvas, 
                        text='UF     '
                        )
                    botaoUFP5C.grid(row=32+j, column=6, padx=20)
                    tbCUFP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCUFP5.grid(row=32+j, column=7, pady=0)
                except:
                    pass

                #CEP Participante 5
                try:
                    botaoCEPP5C = Label(
                        canvas, 
                        text='CEP'
                        )
                    botaoCEPP5C.grid(row=32+j, column=8, padx=20)
                    tbCCEPP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCCEPP5.grid(row=32+j, column=9, pady=0)
                except:
                    pass

                #Telefone Participante 5
                try:
                    botaoTelefoneP5C = Label(
                        canvas, 
                        text='Telefone'
                        )
                    botaoTelefoneP5C.grid(row=33+j, column=0, padx=20)
                    tbCTelefoneP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCTelefoneP5.grid(row=33+j, column=1, pady=0)
                except:
                    pass

                #Email Participante 5
                try:
                    botaoEmailP5C = Label(
                        canvas, 
                        text='Email'
                        )
                    botaoEmailP5C.grid(row=33+j, column=2, padx=20)
                    tbCEmailP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCEmailP5.grid(row=33+j, column=3, pady=0)
                except:
                    pass

                #Tipo de Operação
                try:
                    tipoOpP5C = Label(
                        canvas, 
                        text='Tipo Operação',
                        )
                    tipoOpP5C.grid(row=33+j, column=4)
                    variableOps5 = StringVar(canvas)
                    tbCTipoOps5 = OptionMenu(canvas, variableOps5, "", "PF", "PJ")
                    tbCTipoOps5.grid(row=33+j, column=5)
                except:
                    pass

                #Sexo Operação
                try:
                    sexoP5C = Label(
                        canvas, 
                        text='Sexo'
                        )
                    sexoP5C.grid(row=33+j, column=6)
                    variableSexo5 = StringVar(canvas)
                    canvasexoP5 = OptionMenu(canvas, variableSexo5, "", "F", "M")
                    canvasexoP5.grid(row=33+j, column=7)
                except:
                    pass

                #Relação
                try:
                    relacaoP5C = Label(
                        canvas, 
                        text='Relação'
                        )
                    relacaoP5C.grid(row=33+j, column=8)
                    variableRelacao5 = StringVar(canvas)
                    wRelacaoP5 = OptionMenu(canvas, variableRelacao5, "", "Titular", "Cônjuge", "Procurador" ,"Pai", "Mãe", "Irmão")
                    wRelacaoP5.grid(row=33+j, column=9)
                except:
                    pass

                #Participação
                try:
                    participacaoP5C = Label(
                        canvas, 
                        text='Participação'
                        )
                    participacaoP5C.grid(row=34+j, column=0, padx=20)
                    tbCparticipacaoP5 = Entry(
                        canvas,
                        width=20
                        )
                    tbCparticipacaoP5.grid(row=34+j, column=1, pady=5)
                except:
                    pass
            except:
                print('Não foi possível inserir o participante 1')
            
            #Adicionar Mais um
            i+= 1

            #Ajuste de Coluna    
            if i>1:
                j=5+j
        
        #Passo 3 - Adicionar a barra de rolagem ao canvas
        scrollbar = Scrollbar(main_frame, orient=VERTICAL, command=my_canvas.yview)
        scrollbar.pack(side=RIGHT, fill=Y)

        #Pegar Valores dos Participantes
        def getValuesParticipantes(numDParticipantes):
            global dadosC
            global dadosI
            global listaFinal
            
            listaFinal = {}
            
            if 1 <=numDParticipantes:
                listaParticipante1 = {"nomeCompletoP1":tbCNomeP1.get(),"cpfP1":tbCCpfP1.get(),"dataNascimentoP1":tbCDataNascimentoP1.get(),"cidadeP1":tbCCidadeP1.get(),"endereçoP1":tbCEnderecodP1.get(),
                "numeroResidenciaP1":tbCNumeroP1.get(),"complementoP1":tbCComplementoP1.get(), "bairroP1":tbCBairroP1.get(),"estadoP1":tbCUFP1.get(),"cepP1":tbCCEPP1.get(),"telefoneP1":tbCTelefoneP1.get(),
                "emailP1":tbCEmailP1.get(),"tipoOperacaoP1":variableOps1.get(),"sexoP1":variableSexo1.get(),"relacaoDoP1":variableRelacao1.get(),"participacaoNaOperacaoP1":tbCparticipacaoP1.get(),
                                    }

                #Concatenar Dicionarios                    
                listaFinal = {**listaFinal, **listaParticipante1}
            
            if 2 <=numDParticipantes:
                listaParticipante2 = {"nomeCompletoP2":tbCNomeP2.get(),"cpfP2":tbCCpfP2.get(),"dataNascimentoP2":tbCDataNascimentoP2.get(),"cidadeP2":tbCCidadeP2.get(),"endereçoP2":tbCEnderecodP2.get(),
                "numeroResidenciaP2":tbCNumeroP2.get(),"complementoP2":tbCComplementoP2.get(), "bairroP2":tbCBairroP2.get(),"estadoP2":tbCUFP2.get(),"cepP2":tbCCEPP2.get(),"telefoneP2":tbCTelefoneP2.get(),
                "emailP2":tbCEmailP2.get(),"tipoOperacaoP2":variableOps2.get(),"sexoP2":variableSexo2.get(),"relacaoDoP2":variableRelacao2.get(),"participacaoNaOperacaoP2":tbCparticipacaoP2.get(),
                                    }

                #Concatenar Dicionarios               
                listaFinal = {**listaFinal, **listaParticipante2}
            
            if 3 <=numDParticipantes:
                listaParticipante3 = {"nomeCompletoP3":tbCNomeP3.get(),"cpfP3":tbCCpfP3.get(),"dataNascimentoP3":tbCDataNascimentoP3.get(),"cidadeP3":tbCCidadeP3.get(),"endereçoP3":tbCEnderecodP3.get(),
                "numeroResidenciaP3":tbCNumeroP3.get(),"complementoP3":tbCComplementoP3.get(), "bairroP3":tbCBairroP3.get(),"estadoP3":tbCUFP3.get(),"cepP3":tbCCEPP3.get(),"telefoneP3":tbCTelefoneP3.get(),
                "emailP3":tbCEmailP3.get(),"tipoOperacaoP3":variableOps3.get(),"sexoP3":variableSexo3.get(),"relacaoDoP3":variableRelacao3.get(),"participacaoNaOperacaoP3":tbCparticipacaoP3.get(),
                                    }
                
                #Concatenar Dicionarios               
                listaFinal = {**listaFinal, **listaParticipante3}

            if 4 <=numDParticipantes:    
                listaParticipante4 = {"nomeCompletoP4":tbCNomeP4.get(),"cpfP4":tbCCpfP4.get(),"dataNascimentoP4":tbCDataNascimentoP4.get(),"cidadeP4":tbCCidadeP4.get(),"endereçoP4":tbCEnderecodP4.get(),
                "numeroResidenciaP4":tbCNumeroP4.get(),"complementoP4":tbCComplementoP4.get(), "bairroP4":tbCBairroP4.get(),"estadoP4":tbCUFP4.get(),"cepP4":tbCCEPP4.get(),"telefoneP4":tbCTelefoneP4.get(),
                "emailP4":tbCEmailP4.get(),"tipoOperacaoP4":variableOps4.get(),"sexoP4":variableSexo4.get(),"relacaoDoP4":variableRelacao4.get(),"participacaoNaOperacaoP4":tbCparticipacaoP4.get(),
                                    }
                
                #Concatenar Dicionarios               
                listaFinal = {**listaFinal, **listaParticipante4}
            
            if 5 <=numDParticipantes:
                listaParticipante5 = {"nomeCompletoP5":tbCNomeP5.get(),"cpfP5":tbCCpfP5.get(),"dataNascimentoP5":tbCDataNascimentoP5.get(),"cidadeP5":tbCCidadeP5.get(),"endereçoP5":tbCEnderecodP5.get(),
                "numeroResidenciaP5":tbCNumeroP5.get(),"complementoP5":tbCComplementoP5.get(), "bairroP5":tbCBairroP5.get(),"estadoP5":tbCUFP5.get(),"cepP5":tbCCEPP5.get(),"telefoneP5":tbCTelefoneP5.get(),
                "emailP5":tbCEmailP5.get(),"tipoOperacaoP5":variableOps5.get(),"sexoP5":variableSexo5.get(),"relacaoDoP5":variableRelacao5.get(),"participacaoNaOperacaoP5":tbCparticipacaoP5.get(),
                                    }
                
                #Concatenar Dicionarios               
                listaFinal = {**listaFinal, **listaParticipante5}
            
            dadosC = getValuesContrato()
            dadosI = getValuesImovel()

            tk.messagebox.showwarning(title='ATENÇÃO!', message='Dados salvos com sucesso!')

        #Salvar Dados
        botaoSalvarDados = Button(
            canvas,
            text ='Salvar Dados',
            command = lambda:getValuesParticipantes(nParticipante)
        )
        botaoSalvarDados.grid(row=34+j+1, column=0, pady=35)

        #Enviar Para Aztronic
        botaoEnviarDados = Button(
            canvas,
            text ='Enviar para Aztronic',
            command = lambda:preencherSelenium()
        )
        botaoEnviarDados.grid(row=34+j+1, column=1, pady=35)

        #Importar Valores
        botaoImportarValores = Button(
        canvas, 
        text ='Importar Valores', 
        command = lambda:preencherDadosParticipantes()
        ) 
        botaoImportarValores.grid(row=29, column=3)

        #Passo 4 - Configurar o canvas
        my_canvas.configure(yscrollcommand=scrollbar.set)
        my_canvas.bind('<Configure>', lambda e:my_canvas.configure(scrollregion=my_canvas.bbox("all")))
    except:  
        pass

#Preencher Valores de Participantes        
def preencherDadosParticipantes():
    global textoParticipantes
    nParticipante = int(tbCNumParticipantes.get())
    
    #Declarar Variávies
    try:
        global tbCNomeP1
        global tbCCpfP1
        global tbCDataNascimentoP1
        global tbCCidadeP1
        global tbCEnderecodP1
        global tbCNumeroP1
        global tbCComplementoP1
        global tbCBairroP1
        global tbCUFP1
        global tbCCEPP1
        global tbCTelefoneP1
        global tbCEmailP1
        global tbCparticipacaoP1
        global tbCNomeP2
        global tbCCpfP2
        global tbCDataNascimentoP2
        global tbCCidadeP2
        global tbCEnderecodP2
        global tbCNumeroP2
        global tbCComplementoP2
        global tbCBairroP2
        global tbCUFP2
        global tbCCEPP2
        global tbCTelefoneP2
        global tbCEmailP2
        global tbCparticipacaoP2
        global tbCNomeP3
        global tbCCpfP3
        global tbCDataNascimentoP3
        global tbCCidadeP3
        global tbCEnderecodP3
        global tbCNumeroP3
        global tbCComplementoP3
        global tbCBairroP3
        global tbCUFP3
        global tbCCEPP3
        global tbCTelefoneP3
        global tbCEmailP3
        global tbCparticipacaoP3
        global tbCNomeP4
        global tbCCpfP4
        global tbCDataNascimentoP4
        global tbCCidadeP4
        global tbCEnderecodP4
        global tbCNumeroP4
        global tbCComplementoP4
        global tbCBairroP4
        global tbCUFP4
        global tbCCEPP4
        global tbCTelefoneP4
        global tbCEmailP4
        global tbCparticipacaoP4
        global tbCNomeP5
        global tbCCpfP5
        global tbCDataNascimentoP5
        global tbCCidadeP5
        global tbCEnderecodP5
        global tbCNumeroP5
        global tbCComplementoP5
        global tbCBairroP5
        global tbCUFP5
        global tbCCEPP5
        global tbCTelefoneP5
        global tbCEmailP5
        global tbCparticipacaoP5
    except:
        pass
    
    #Participante 1
    try:
        if 1 <= nParticipante:
            #Excluir Dados Participante1
            tbCNomeP1.delete(0, 'end')
            tbCCpfP1.delete(0, 'end')
            tbCDataNascimentoP1.delete(0, 'end')
            tbCCidadeP1.delete(0, 'end')
            tbCEnderecodP1.delete(0, 'end')
            tbCNumeroP1.delete(0, 'end')
            tbCComplementoP1.delete(0, 'end')
            tbCBairroP1.delete(0, 'end')
            tbCUFP1.delete(0, 'end')
            tbCCEPP1.delete(0, 'end')
            tbCTelefoneP1.delete(0, 'end')
            tbCEmailP1.delete(0, 'end')
            tbCparticipacaoP1.delete(0, 'end')

            #Inserir Valores Participante 1
            tbCNomeP1.insert(0, textoParticipantes['nome1'])
            tbCCpfP1.insert(0, textoParticipantes['cpf1'])
            tbCDataNascimentoP1.insert(0, textoParticipantes['data de nascimento1'])
            tbCCidadeP1.insert(0, textoParticipantes['cidade1'])
            tbCEnderecodP1.insert(0, ''.join([d for d in textoParticipantes['endereço1'] if d.isalpha() or d==" "]).strip())
            tbCNumeroP1.insert(0, ''.join([d for d in textoParticipantes['endereço1'] if d.isdigit()]))
            tbCComplementoP1.insert(0, textoParticipantes['complemento1'])
            tbCBairroP1.insert(0, textoParticipantes['bairro1'])
            tbCUFP1.insert(0, textoParticipantes['uf1'])
            tbCCEPP1.insert(0, textoParticipantes['cep1'])
            tbCTelefoneP1.insert(0, textoParticipantes['telefone1'])
            tbCEmailP1.insert(0, textoParticipantes['email1'])
            tbCparticipacaoP1.insert(0, str(textoParticipantes['participação1']).replace("%",""))
    except:
        print('Erro ao importar Participante 1')
    
    #Participante 2
    try:
        if 2 <= nParticipante:
            #Excluir Dados Participante2
            tbCNomeP2.delete(0, 'end')
            tbCCpfP2.delete(0, 'end')
            tbCDataNascimentoP2.delete(0, 'end')
            tbCCidadeP2.delete(0, 'end')
            tbCEnderecodP2.delete(0, 'end')
            tbCNumeroP2.delete(0, 'end')
            tbCComplementoP2.delete(0, 'end')
            tbCBairroP2.delete(0, 'end')
            tbCUFP2.delete(0, 'end')
            tbCCEPP2.delete(0, 'end')
            tbCTelefoneP2.delete(0, 'end')
            tbCEmailP2.delete(0, 'end')
            tbCparticipacaoP2.delete(0, 'end')

            #Inserir Valores Participante 2
            tbCNomeP2.insert(0, textoParticipantes['nome2'])
            tbCCpfP2.insert(0, textoParticipantes['cpf2'])
            tbCDataNascimentoP2.insert(0, textoParticipantes['data de nascimento2'])
            tbCCidadeP2.insert(0, textoParticipantes['cidade2'])
            tbCEnderecodP2.insert(0, ''.join([d for d in textoParticipantes['endereço2'] if d.isalpha() or d==" "]).strip())
            tbCNumeroP2.insert(0, ''.join([d for d in textoParticipantes['endereço2'] if d.isdigit()]))
            tbCComplementoP2.insert(0, textoParticipantes['complemento2'])
            tbCBairroP2.insert(0, textoParticipantes['bairro2'])
            tbCUFP2.insert(0, textoParticipantes['uf2'])
            tbCCEPP2.insert(0, textoParticipantes['cep2'])
            tbCTelefoneP2.insert(0, textoParticipantes['telefone2'])
            tbCEmailP2.insert(0, textoParticipantes['email2'])
            tbCparticipacaoP2.insert(0, str(textoParticipantes['participação2']).replace("%",""))   
    except:
        print('Erro ao importar Participante 2')
    
    #Participante 3
    try:
        if 3 <= nParticipante:
            #Excluir Dados Participante3
            tbCNomeP3.delete(0, 'end')
            tbCCpfP3.delete(0, 'end')
            tbCDataNascimentoP3.delete(0, 'end')
            tbCCidadeP3.delete(0, 'end')
            tbCEnderecodP3.delete(0, 'end')
            tbCNumeroP3.delete(0, 'end')
            tbCComplementoP3.delete(0, 'end')
            tbCBairroP3.delete(0, 'end')
            tbCUFP3.delete(0, 'end')
            tbCCEPP3.delete(0, 'end')
            tbCTelefoneP3.delete(0, 'end')
            tbCEmailP3.delete(0, 'end')
            tbCparticipacaoP3.delete(0, 'end')

            #Inserir Valores Participante 3
            tbCNomeP3.insert(0, textoParticipantes['nome3'])
            tbCCpfP3.insert(0, textoParticipantes['cpf3'])
            tbCDataNascimentoP3.insert(0, textoParticipantes['data de nascimento3'])
            tbCCidadeP3.insert(0, textoParticipantes['cidade3'])
            tbCEnderecodP3.insert(0, ''.join([d for d in textoParticipantes['endereço3'] if d.isalpha() or d==" "]).strip())
            tbCNumeroP3.insert(0, ''.join([d for d in textoParticipantes['endereço3'] if d.isdigit()]))
            tbCComplementoP3.insert(0, textoParticipantes['complemento3'])
            tbCBairroP3.insert(0, textoParticipantes['bairro3'])
            tbCUFP3.insert(0, textoParticipantes['uf3'])
            tbCCEPP3.insert(0, textoParticipantes['cep3'])
            tbCTelefoneP3.insert(0, textoParticipantes['telefone3'])
            tbCEmailP3.insert(0, textoParticipantes['email3'])
            tbCparticipacaoP3.insert(0, str(textoParticipantes['participação3']).replace("%",""))
    except:
        print('Erro ao importar Participante 3')
    
    #Participante 4
    try:
        if 4 <= nParticipante:
            #Excluir Dados Participante4
            tbCNomeP4.delete(0, 'end')
            tbCCpfP4.delete(0, 'end')
            tbCDataNascimentoP4.delete(0, 'end')
            tbCCidadeP4.delete(0, 'end')
            tbCEnderecodP4.delete(0, 'end')
            tbCNumeroP4.delete(0, 'end')
            tbCComplementoP4.delete(0, 'end')
            tbCBairroP4.delete(0, 'end')
            tbCUFP4.delete(0, 'end')
            tbCCEPP4.delete(0, 'end')
            tbCTelefoneP4.delete(0, 'end')
            tbCEmailP4.delete(0, 'end')
            tbCparticipacaoP4.delete(0, 'end')

            #Inserir Valores Participante 4
            tbCNomeP4.insert(0, textoParticipantes['nome4'])
            tbCCpfP4.insert(0, textoParticipantes['cpf4'])
            tbCDataNascimentoP4.insert(0, textoParticipantes['data de nascimento4'])
            tbCCidadeP4.insert(0, textoParticipantes['cidade4'])
            tbCEnderecodP4.insert(0, ''.join([d for d in textoParticipantes['endereço4'] if d.isalpha() or d==" "]).strip())
            tbCNumeroP4.insert(0, ''.join([d for d in textoParticipantes['endereço4'] if d.isdigit()]))
            tbCComplementoP4.insert(0, textoParticipantes['complemento4'])
            tbCBairroP4.insert(0, textoParticipantes['bairro4'])
            tbCUFP4.insert(0, textoParticipantes['uf4'])
            tbCCEPP4.insert(0, textoParticipantes['cep4'])
            tbCTelefoneP4.insert(0, textoParticipantes['telefone4'])
            tbCEmailP4.insert(0, textoParticipantes['email4'])
            tbCparticipacaoP4.insert(0, str(textoParticipantes['participação4']).replace("%",""))
    except:
        print('Erro ao importar Participante 4')

    #Participante 5
    try:
        if 5 <= nParticipante:
            #Excluir Dados Participante5
            tbCNomeP5.delete(0, 'end')
            tbCCpfP5.delete(0, 'end')
            tbCDataNascimentoP5.delete(0, 'end')
            tbCCidadeP5.delete(0, 'end')
            tbCEnderecodP5.delete(0, 'end')
            tbCNumeroP5.delete(0, 'end')
            tbCComplementoP5.delete(0, 'end')
            tbCBairroP5.delete(0, 'end')
            tbCUFP5.delete(0, 'end')
            tbCCEPP5.delete(0, 'end')
            tbCTelefoneP5.delete(0, 'end')
            tbCEmailP5.delete(0, 'end')
            tbCparticipacaoP5.delete(0, 'end')

            #Inserir Valores Participante 5
            tbCNomeP5.insert(0, textoParticipantes['nome5'])
            tbCCpfP5.insert(0, textoParticipantes['cpf5'])
            tbCDataNascimentoP5.insert(0, textoParticipantes['data de nascimento5'])
            tbCCidadeP5.insert(0, textoParticipantes['cidade5'])
            tbCEnderecodP5.insert(0, ''.join([d for d in textoParticipantes['endereço5'] if d.isalpha() or d==" "]).strip())
            tbCNumeroP5.insert(0, ''.join([d for d in textoParticipantes['endereço5'] if d.isdigit()]))
            tbCComplementoP5.insert(0, textoParticipantes['complemento5'])
            tbCBairroP5.insert(0, textoParticipantes['bairro5'])
            tbCUFP5.insert(0, textoParticipantes['uf5'])
            tbCCEPP5.insert(0, textoParticipantes['cep5'])
            tbCTelefoneP5.insert(0, textoParticipantes['telefone5'])
            tbCEmailP5.insert(0, textoParticipantes['email5'])
            tbCparticipacaoP5.insert(0, str(textoParticipantes['participação5']).replace("%",""))
    except:
        print('Erro ao importar Participante 5')

#Preencher API Selenium
def preencherSelenium():
    #preencherAPI(calculoFluxo, dadosC, dadosI, listaFinal, produto)
    preencherAPI(calculoFluxo,dadosC,dadosI,listaFinal,produto)

######## Botões relacionados ao Contrato ########
textBoxContrato = Entry(
    canvas, 
    width=30
    )
textBoxContrato.grid(row=0, column=2)    
botaoContrato = Label(
    canvas, 
    text='Selecione o contrato em PDF'
    )
botaoContrato.grid(row=0, column=0, padx=10)
anexarContrato = Button(
    canvas, 
    text ='Selecione o Contrato', 
    command = lambda:open_file_pdf(textBoxContrato)
    ) 
anexarContrato.grid(row=0, column=1)

####### Botão Cálculo Fluxo #######
textBoxCalculoFluxo = Entry(
    canvas, 
    width=30
    )
textBoxCalculoFluxo.grid(row=0, column=6)    
botaoCalculoFluxo = Label(
    canvas, 
    text='Selecione a planilha de Cálculo Fluxo'
    )
botaoCalculoFluxo.grid(row=0, column=4, padx=10)
anexarCalculoFluxo = Button(
    canvas, 
    text ='Selecionar o Cálculo Fluxo', 
    command = lambda:open_file_excel(textBoxCalculoFluxo)
    ) 
anexarCalculoFluxo.grid(row=0, column=5)

####### Taxa de Juros #######
textBoxTaxaJuros = Entry(
    canvas, 
    width=30
    )
textBoxTaxaJuros.grid(row=2, column=6)    
botaoCalculoFluxo = Label(
    canvas, 
    text='Taxa de Juros'
    )
botaoCalculoFluxo.grid(row=2, column=5, padx=10)


######## Botões Simulador ########
textBoxSimulador = Entry(
    canvas,
    width=30,
    
    )
textBoxSimulador.grid(row=1, column=2)    
botaoSimulador = Label(
    canvas, 
    text='Selecione o simulador em Excel'
    )
botaoSimulador.grid(row=1, column=0, padx=10)
anexarSimulador = Button(
    canvas, 
    text ='Selecione o Simulador', 
    command = lambda:open_file_excel(textBoxSimulador)
    ) 
anexarSimulador.grid(row=1, column=1)

######## Botões Laudo ########
textBoxLaudo = Entry(
    canvas,
    width=30
    )
textBoxLaudo.grid(row=2, column=2)
botaoLaudo = Label(
    canvas, 
    text='Selecione o Laudo em PDF'
    )
botaoLaudo.grid(row=2, column=0, padx=10)
anexarLaudo = Button(
    canvas, 
    text ='Selecione o Laudo', 
    command = lambda:open_file_pdf(textBoxLaudo)
    ) 
anexarLaudo.grid(row=2, column=1)

######## Botão Importar Valores ########
upld = tk.Button(
    canvas, 
    text='Importar Valores', 
    command=uploadFiles
    )
upld.grid(row=3, columnspan=1, pady=10)

######### Botão Validar ########
conf = Button(
    canvas, 
    text='Validar Valores', 
    command=validarEntry
    )
conf.grid(row=3, columnspan=5, pady=10)

######## Menu Dados da Operação ########
dadosOperacao = Label(
    canvas, 
    text='Dados da Operação',
    font=("bold",15),
    )
dadosOperacao.grid(row=6, column=0, pady=10)

######## Menu Superior Dados da Operação ########
dadosContrato = Label(
    canvas, 
    text='Contrato',
    font=40
    )
dadosContrato.grid(row=7, column=0, pady=10)
dadosSimulador = Label(
    canvas, 
    text='Simulador',
    font=40,
    )
dadosSimulador.grid(row=7, column=4, padx=30)
dadosImovelLaudo = Label(
    canvas, 
    text='Dados do Imóvel - Laudo',
    font=40,
    )
dadosImovelLaudo.grid(row=7, column=7, padx=30)

######## Menu Inferior Dados do Imóvel ########
dadosImovel = Label(
    canvas, 
    text='Dados do imóvel',
    font=("bold",15),
    )
dadosImovel.grid(row=23, column=0, padx=30, pady=30)
dadosLaudo = Label(
    canvas, 
    text='Dados do Laudo',
    font=40,
    )
dadosLaudo.grid(row=23, column=7, padx=30)

######## Menu Participantes da Operação ########
dadosParticipantesOps = Label(
    canvas, 
    text='Participantes da Operação',
    font=("bold",15),
    )
dadosParticipantesOps.grid(row=28, column=0, padx=30, pady=30)

######## Saída de Dados Contratos ########
try:
    #Valor Bruto
    try:
        botaovalorBrutoC = Label(
            canvas, 
            text='Valor Bruto'
            )
        botaovalorBrutoC.grid(row=9, column=0)
        tbCValorBruto = Entry(
            canvas,
            width=20
            )
        tbCValorBruto.grid(row=9, column=1)
    except:
        pass

    #Valor Liquido
    try:
        botaovalorLiquidoC = Label(
            canvas, 
            text='Valor Liquido'
            )
        botaovalorLiquidoC.grid(row=10, column=0)
        tbCValorLiquido = Entry(
            canvas,
            width=20
            )
        tbCValorLiquido.grid(row=10, column=1)
    except:
        pass

    #Sistena de Amortização
    try:
        botaoSistemaAmortizacaoC = Label(
            canvas, 
            text='Sistema de Amortização'
            )
        botaoSistemaAmortizacaoC.grid(row=11, column=0)
        tbCAmortizacao = Entry(
            canvas,
            width=20
            )
        tbCAmortizacao.grid(row=11, column=1)
    except:
        pass

    #Índice de Correção
    try:
        botaoIndiceC = Label(
            canvas, 
            text='Indice de Correção'
            )
        botaoIndiceC.grid(row=12, column=0)
        tbCIndice = Entry(
            canvas,
            width=20
            )
        tbCIndice.grid(row=12, column=1)
    except:
        pass

    #Prazo
    try:
        botaoPrazoC = Label(
            canvas, 
            text='Prazo da Operação'
            )
        botaoPrazoC.grid(row=13, column=0)
        tbCPrazo = Entry(
            canvas,
            width=20
            )
        tbCPrazo.grid(row=13, column=1)
    except:
        pass

    #Número de Parcelas
    try:
        botaoParcelasC = Label(
            canvas, 
            text='Número de parcelas'
            )
        botaoParcelasC.grid(row=14, column=0)
        tbCParcelas = Entry(
            canvas,
            width=20
            )
        tbCParcelas.grid(row=14, column=1)
    except:
        pass

    #Data Desembolso
    try:
        botaoDataDesemC = Label(
            canvas, 
            text='Data de Desembolso'
            )
        botaoDataDesemC.grid(row=15, column=0)
        tbCDataDesem = Entry(
            canvas,
            width=20
            )
        tbCDataDesem.grid(row=15, column=1)
    except:
        pass

    #Data Primeira Parcela
    try:
        botaoDataPriParcelaC = Label(
            canvas, 
            text='Data Primeira Parcela'
            )
        botaoDataPriParcelaC.grid(row=16, column=0)
        tbCDataPriParcela = Entry(
            canvas,
            width=20
            )
        tbCDataPriParcela.grid(row=16, column=1)
    except:
        pass

    #Data Última Parcela
    try:
        botaoDataUltimaParcelaC = Label(
            canvas, 
            text='Data Última Parcela'
            )
        botaoDataUltimaParcelaC.grid(row=17, column=0)
        tbCDataUltimaParcela = Entry(
            canvas,
            width=20
            )
        tbCDataUltimaParcela.grid(row=17, column=1)
    except:
        pass

    #Data Valor Primeira Parcela
    try:
        botaoValorPriParC = Label(
            canvas, 
            text='Valor Primeira Parcela (Juros + Amortização)'
            )
        botaoValorPriParC.grid(row=18, column=0)
        tbCValorPriPar = Entry(
            canvas,
            width=20
            )
        tbCValorPriPar.grid(row=18, column=1)
    except:
        pass

    #Data Valor Primeira Parcela + Encargos
    try:
        botaoValorPriParEncargosC = Label(
            canvas, 
            text='Valor Primeira Parcela'
            )
        botaoValorPriParEncargosC.grid(row=19, column=0)
        tbCValorPriEncargosPar = Entry(
            canvas,
            width=20
            )
        tbCValorPriEncargosPar.grid(row=19, column=1)
    except:
        pass

    #Data Contrato
    try:
        botaoDataContratoC = Label(
            canvas, 
            text='Data do Contrato'
            )
        botaoDataContratoC.grid(row=20, column=0)
        tbCDataContrato = Entry(
            canvas,
            width=20
            )
        tbCDataContrato.grid(row=20, column=1)
    except:
        pass

    #Valor de Avaliação do Imóvel
    try:
        botaoValorAvaliacaoC = Label(
            canvas, 
            text='Valor de Avaliação do Imóvel'
            )
        botaoValorAvaliacaoC.grid(row=24, column=0)
        tbCValorAvaliacao = Entry(
            canvas,
            width=20
            )
        tbCValorAvaliacao.grid(row=24, column=1)
    except:
        pass

    #Número da Matrícula
    try:
        botaoMatriculaC = Label(
            canvas, 
            text='Matrícula do Imóvel'
            )
        botaoMatriculaC.grid(row=25, column=0)
        tbCMatricula = Entry(
            canvas,
            width=20
            )
        tbCMatricula.grid(row=25, column=1)
    except:
        pass

    #Cartório
    try:
        botaoCartorioC = Label(
            canvas, 
            text='Cartório'
            )
        botaoCartorioC.grid(row=26, column=0)
        tbCCartorio = Entry(
            canvas,
            width=20
            )
        tbCCartorio.grid(row=26, column=1)
    except:
        pass
except:
    print("Erro ao gerar menu contrato")

######## Saída de Dados Simulador ########
try:
    #Valor Bruto
    try:
        botaovalorBrutoS = Label(
            canvas, 
            text='Valor Bruto'
            )
        botaovalorBrutoS.grid(row=9, column=4)
        tbSValorBruto = Entry(
            canvas,
            width=20
            )
        tbSValorBruto.grid(row=9, column=5)
    except:
        pass

    #Valor Liquido
    try:
        botaovalorLiquidoS = Label(
            canvas, 
            text='Valor Liquido'
            )
        botaovalorLiquidoS.grid(row=10, column=4)
        tbSValorLiquido = Entry(
            canvas,
            width=20
            )
        tbSValorLiquido.grid(row=10, column=5)
    except:
        pass

    #Sistena de Amortização
    try:
        botaoSistemaAmortizacaoS = Label(
            canvas, 
            text='Sistema de Amortização'
            )
        botaoSistemaAmortizacaoS.grid(row=11, column=4)
        tbSAmortizacao = Entry(
            canvas,
            width=20
            )
        tbSAmortizacao.grid(row=11, column=5)
    except:
        pass

    #Índice de Correção
    try:
        botaoIndiceS = Label(
            canvas, 
            text='Indice de Correção'
            )
        botaoIndiceS.grid(row=12, column=4)
        tbSIndice = Entry(
            canvas,
            width=20
            )
        tbSIndice.grid(row=12, column=5)
    except:
        pass

    #Prazo
    try:
        botaoPrazoS = Label(
            canvas, 
            text='Prazo da Operação'
            )
        botaoPrazoS.grid(row=13, column=4)
        tbSPrazo = Entry(
            canvas,
            width=20
            )
        tbSPrazo.grid(row=13, column=5)
    except:
        pass

    #Número de Parcelas
    try:
        botaoParcelasS = Label(
            canvas, 
            text='Número de parcelas'
            )
        botaoParcelasS.grid(row=14, column=4)
        tbSParcelas = Entry(
            canvas,
            width=20
            )
        tbSParcelas.grid(row=14, column=5)
    except:
        pass

    #Data Desembolso
    try:
        botaoDataDesemS = Label(
            canvas, 
            text='Data de Desembolso'
            )
        botaoDataDesemS.grid(row=15, column=4)
        tbSDataDesem = Entry(
            canvas,
            width=20
            )
        tbSDataDesem.grid(row=15, column=5)
    except:
        pass

    #Data Primeira Parcela
    try:
        botaoDataPriParcelaS = Label(
            canvas, 
            text='Data Primeira Parcela'
            )
        botaoDataPriParcelaS.grid(row=16, column=4)
        tbSDataPriParcela = Entry(
            canvas,
            width=20
            )
        tbSDataPriParcela.grid(row=16, column=5)
    except:
        pass

    #Data Última Parcela
    try:
        botaoDataUltimaParcelaS = Label(
            canvas, 
            text='Data Última Parcela'
            )
        botaoDataUltimaParcelaS.grid(row=17, column=4)
        tbSDataUltimaParcela = Entry(
            canvas,
            width=20
            )
        tbSDataUltimaParcela.grid(row=17, column=5)
    except:
        pass

    #Data Valor Primeira Parcela
    try:
        botaoValorPriParS = Label(
            canvas, 
            text='Valor Primeira Parcela (Juros + Amortização)'
            )
        botaoValorPriParS.grid(row=18, column=4)
        tbSValorPriPar = Entry(
            canvas,
            width=20
            )
        tbSValorPriPar.grid(row=18, column=5)
    except:
        pass

    #Data Valor Primeira Parcela + Encargos
    try:
        botaoValorPriParEncargosS = Label(
            canvas, 
            text='Valor Primeira Parcela'
            )
        botaoValorPriParEncargosS.grid(row=19, column=4)
        tbSValorPriEncargosPar = Entry(
            canvas,
            width=20
            )
        tbSValorPriEncargosPar.grid(row=19, column=5)
    except:
        pass

    #Valor de Avaliação do Imóvel
    try:
        botaoValorAvaliacaoS = Label(
            canvas, 
            text='Valor de Avaliação do Imóvel'
            )
        botaoValorAvaliacaoS.grid(row=24, column=4)
        tbSValorAvaliacao = Entry(
            canvas,
            width=20
            )
        tbSValorAvaliacao.grid(row=24, column=5)
    except:
        pass
except:
    print("Erro ao gerar menu contrato")

######## Saída de Dados Laudo ########
try:
    #Valor de Avaliação do Imóvel
    try:
        botaoValorAvaliacaoL = Label(
            canvas, 
            text='Valor de Avaliação do Imóvel'
            )
        botaoValorAvaliacaoL.grid(row=24, column=7, padx=40)
        tbLValorAvaliacao = Entry(
            canvas,
            width=20
            )
        tbLValorAvaliacao.grid(row=24, column=8)
    except:
        pass

    #Número da Matrícula
    try:
        botaoMatriculaL = Label(
            canvas, 
            text='Matrícula do Imóvel'
            )
        botaoMatriculaL.grid(row=25, column=7)
        tbLMatricula = Entry(
            canvas,
            width=20
            )
        tbLMatricula.grid(row=25, column=8)
    except:
        pass

    #Cartório
    try:
        botaoCartorioL = Label(
            canvas, 
            text='Cartório'
            )
        botaoCartorioL.grid(row=26, column=7)
        tbLCartorio = Entry(
            canvas,
            width=20
            )
        tbLCartorio.grid(row=26, column=8)
    except:
        pass
except:
    print("Erro ao gerar menu contrato")

######## Saída de Dados Do Imóvel By Laudo ########
try:
    #Endereço
    try:
        botaoEndereco = Label(
            canvas, 
            text='Endereço Imóvel'
            )
        botaoEndereco.grid(row=9, column=7, padx=40)
        tbLEndereco = Entry(
            canvas,
            width=20
            )
        tbLEndereco.grid(row=9, column=8)
    except:
        pass
    
    #Número da residência
    try:
        botaoNumeroResidencia = Label(
            canvas, 
            text='Número da residência'
            )
        botaoNumeroResidencia.grid(row=10, column=7, padx=40)
        tbLNumeroEndereco = Entry(
            canvas,
            width=20
            )
        tbLNumeroEndereco.grid(row=10, column=8)
    except:
        pass
    
    #Complemento
    try:
        botaoComplemento = Label(
            canvas, 
            text='Complemento'
            )
        botaoComplemento.grid(row=11, column=7, padx=40)
        tbLComplemento = Entry(
            canvas,
            width=20
            )
        tbLComplemento.grid(row=11, column=8)
    except:
        pass
    
    #Bairro
    try:
        botaoBairro = Label(
            canvas, 
            text='Bairro'
            )
        botaoBairro.grid(row=12, column=7, padx=40)
        tbLBairro = Entry(
            canvas,
            width=20
            )
        tbLBairro.grid(row=12, column=8)
    except:
        pass
    
    #CEP
    try:
        botaoCEP = Label(
            canvas, 
            text='CEP'
            )
        botaoCEP.grid(row=13, column=7, padx=40)
        tbLCEP = Entry(
            canvas,
            width=20
            )
        tbLCEP.grid(row=13, column=8)
    except:
        pass
    
    #Cidade
    try:
        botaoCidade = Label(
            canvas, 
            text='Cidade'
            )
        botaoCidade.grid(row=14, column=7, padx=40)
        tbLCidade = Entry(
            canvas,
            width=20
            )
        tbLCidade.grid(row=14, column=8)
    except:
        pass
    
    #Estado
    try:
        botaoUF = Label(
            canvas, 
            text='Estado'
            )
        botaoUF.grid(row=15, column=7, padx=40)
        tbLUF = Entry(
            canvas,
            width=20
            )
        tbLUF.grid(row=15, column=8)
    except:
        pass
    
    #Unidade
    try:
        botaoUnidade = Label(
            canvas, 
            text='Unidade'
            )
        botaoUnidade.grid(row=16, column=7, padx=40)
        tbLUnidade = Entry(
            canvas,
            width=20
            )
        tbLUnidade.grid(row=16, column=8)
    except:
        pass
    
    #Bloco
    try:
        botaoBloco = Label(
            canvas, 
            text='Bloco'
            )
        botaoBloco.grid(row=17, column=7, padx=40)
        tbLBloco = Entry(
            canvas,
            width=20
            )
        tbLBloco.grid(row=17, column=8)
    except:
        pass
except:
    pass

######## Saída de dados dos participantes ########

#Número de Participantes
try:
    botaoNumeroParticipantes = Label(
        canvas, 
        text='Número de Participantes'
        )
    botaoNumeroParticipantes.grid(row=29, column=0, padx=20,pady=5)
    tbCNumParticipantes = Entry(
        canvas,
        width=20
        )
    tbCNumParticipantes.grid(row=29, column=1)
    
    #Criar Campos
    botaoPuxarParticipantes = Button(
    canvas, 
    text ='Importar Participantes', 
    command = lambda:camposParcipantes()
    ) 
    botaoPuxarParticipantes.grid(row=29, column=2)  
except:
    pass



#Rodar Programa
canvas.mainloop()

37
{'contrato': {'codigo_integracao': '0003045601', 'pep': '', 'id_projeto': '', 'id_tipocontrato': '15', 'id_tipovenda': '1', 'valor_contrato': 272099.1707828351, 'data_contrato': '05/12/2022', 'valor_venda': '764000,00', 'taxa_multa': '2', 'taxa_mora': '1', 'venda_no_estado': 'S', 'score_automatico': '80', 'score_manual': '85', 'seguro_mip': 0.021, 'seguro_dfi': 0.0037, 'taxa_administracao': 25, 'unidade': {'codigo_integracao': '0003045601', 'rua': 'Rua Vinte e Seis de Agosto - de 966/967 ao fim', 'numero': '2079', 'complemento': 'N/A', 'cidade': 'Campo Grande', 'bairro': 'Amambaí', 'UF': 'MS', 'cep': '79005-030', 'cnpj_SPE': '32.402.502/0001-35', 'SPE': 'QI SOCIEDADE DE CREDITO DIRETO S.A.', 'cartorio': '', 'valor_avaliacao': '764000,00', 'unidade': '2079', 'Bloco': 'ÚNICO', 'matricula': '54718', 'numero_cartorio': 0, 'data_habitese': '01/01/2000'}, 'participante': [{'relacao': 'Cônjuge', 'nome': 'VICENTE GARCEZ CHAVES', 'email': 'vicentegarcezchaves@icloud.com', 'participacao': '0'